### Configuração inicial

In [ ]:
from google.cloud import storage
from google.cloud import bigquery
import pandas as pd
import polars as pl
import os
import io
import tarfile
import gc

# Define o diretório para raiz dos arquivos
diretorio = "D:\\__case_ifood"
#  Define o caminho para a chave da conta de serviço
service_account_path = os.path.join(diretorio,"case-ifood-fsg-6f1d7cf34e08.json")
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_path

# Define o diretório para salvar os arquivos baixados do Bucket
diretorio_arquivos_bucket = os.path.join(diretorio,"bases_dados")

# Nome do bucket e do projeto
bucket_name = "case_ifood_fsg"
id_projeto = "case-ifood-fsg"

client = storage.Client()

client_bq = bigquery.Client(project=id_projeto)


### Função para salvar os dados no BigQuery

In [ ]:
def criar_dataset_e_tabela(df, dataset_nome, tabela_nome, client, project_id, location="southamerica-east1"):
    dataset_id = f"{project_id}.{dataset_nome}"
    table_id = f"{dataset_id}.{tabela_nome}"

    try:
        dataset = bigquery.Dataset(dataset_id)
        dataset.location = location
        client.create_dataset(dataset, exists_ok=True)
        print(f"Dataset '{dataset_id}' pronto.")
    except Exception as e:
        print(f"Erro ao criar dataset: {e}")
        return

    if isinstance(df, pl.DataFrame):
        df = df.to_pandas()
        print("Convertido de Polars para Pandas.")

    if not isinstance(df, pd.DataFrame):
        print("O objeto fornecido não é um DataFrame válido.")
        return

    try:
        job = client.load_table_from_dataframe(df, table_id)
        job.result()
        print(f"Tabela '{table_id}' criada com {job.output_rows} linhas.")
    except Exception as e:
        print(f"Erro ao criar tabela: {e}")

### Carrega o arquivo restaurant.csv na camada Bronze do BQ como 'merchants'

In [ ]:
# Define o nome do arquivo
file_name_merchants = "restaurant.csv.gz" 

# Acessa o bucket e o blob (arquivo)
bucket_merchants = client.bucket(bucket_name)
blob_merchants = bucket_merchants.blob(file_name_merchants)

# Faz o download como bytes
data_bytes_merchants = blob_merchants.download_as_bytes()

df_merchants = pd.read_csv(io.BytesIO(data_bytes_merchants), compression='gzip')

# Visualiza as primeiras linhas
df_merchants.head(3)

# Insere os dados no BQ
criar_dataset_e_tabela(
    df=df_merchants,
    dataset_nome="bronze",
    tabela_nome="merchants",
    client=client_bq,
    project_id=id_projeto
)

del file_name_merchants
del bucket_merchants
del blob_merchants
del data_bytes_merchants
del df_merchants
gc.collect()

### Carrega o arquivo consumer.csv na camada Bronze do BQ como 'consumer'

In [ ]:
file_name_consumer = "consumer.csv.gz" 

# Acessa o bucket e o blob (arquivo)
bucket_consumer = client.bucket(bucket_name)
blob_consumer = bucket_consumer.blob(file_name_consumer)

# Faz o download como bytes
data_bytes_consumer = blob_consumer.download_as_bytes()

df_consumer = pd.read_csv(io.BytesIO(data_bytes_consumer), compression='gzip')

# Visualiza as primeiras linhas
print(df_consumer.head(3))

# Insere os dados no BQ
criar_dataset_e_tabela(
    df=df_consumer,
    dataset_nome="bronze",
    tabela_nome="consumer",
    client=client_bq,
    project_id=id_projeto
)

del file_name_consumer
del bucket_consumer
del blob_consumer
del data_bytes_consumer
del df_consumer

gc.collect()

### Carrega o arquivo ab_test_ref.csv na camada Bronze do BQ como ab_test

In [ ]:
file_name_ab_test = "ab_test_ref.tar.gz" 

# Acessa o bucket e o blob (arquivo)
bucket_ab_test = client.bucket(bucket_name)
blob_ab_test = bucket_ab_test.blob(file_name_ab_test)

# Faz o download como bytes
data_bytes_ab_test = blob_ab_test.download_as_bytes()

# Salva o arquivo na máquina local
blob_ab_test.download_to_filename(os.path.join(diretorio_arquivos_bucket,file_name_ab_test))
print(f"Arquivo salvo em: {diretorio_arquivos_bucket}")

del bucket_ab_test
del blob_ab_test
del data_bytes_ab_test

In [ ]:
nome_arquivo_leitura = "ab_test_ref.csv"

# Extrai tudo
with tarfile.open(os.path.join(diretorio_arquivos_bucket,file_name_ab_test), "r:gz") as tar:
    tar.extractall(path=diretorio_arquivos_bucket)

# Faz a leitura do arquivo que possui os dados
df_ab_test = pd.read_csv(os.path.join(diretorio_arquivos_bucket,nome_arquivo_leitura), encoding="utf-8")

print(df_ab_test.head(3))

# Insere os dados no BQ
criar_dataset_e_tabela(
    df=df_ab_test,
    dataset_nome="bronze",
    tabela_nome="ab_test",
    client=client_bq,
    project_id=id_projeto
)

del nome_arquivo_leitura
del file_name_ab_test
del df_ab_test
gc.collect()

### Para o arquivo order.json.gz não foi possível fazer o insert com processamento local, por restrições de memória, com isso, foi criado a tabela diretamente no BQ na camada bronze, chamada orders